# Programming Test 180607
## 20121229 Jun Pyo Park

In [15]:
import pandas as pd
import numpy as np
from scipy.stats import norm

## Parameter Setting

In [16]:
Price = 10
Rate = 0.03
sigma = 0.35
Strike = 10

In [17]:
Time = 1
L = 12
dt = Time / L

## Standard Monte Carlo Simulation

In [18]:
# number of samples
M = 2 ** (np.arange(7) + 10)

### Arithmetic Asian Put Tutorial

In [19]:
paths = pd.DataFrame(np.ones(5) * Price)

In [20]:
# Construct the Path
for j in range(L):
    z = np.random.randn(5)
    paths[j+1] = paths[j] * np.exp((Rate - 0.5 * sigma * sigma ) * dt + sigma * np.sqrt(dt) * z)

In [21]:
f = paths.loc[:,1:]
f

,1,2,3,4,5,6,7,8,9,10,11,12
0,9.868823,9.533708,8.236977,7.622464,8.110069,8.874465,8.711180,7.843142,6.850226,7.523037,7.011537,6.624097
1,9.564724,8.638569,10.620038,9.548234,9.196898,8.096801,6.744560,7.408455,8.560481,7.674999,9.066900,8.744053
2,9.395406,8.177188,8.080832,7.198461,7.100110,6.784145,7.990025,8.224161,10.221478,10.183448,8.952819,8.901669
3,10.411659,9.735496,8.019142,8.175615,6.895835,7.026632,7.750883,7.063488,6.689812,5.668864,6.593407,6.211912
4,10.168871,10.934815,10.285601,10.766457,11.013949,11.618472,11.152151,11.968315,11.135692,10.293233,12.373544,13.462007


In [22]:
final_value = f.mean(axis=1)

In [23]:
final_value

0     8.067477
1     8.655393
2     8.434145
3     7.520229
4    11.264426
dtype: float64

In [24]:
final_value.apply(lambda x : max(Strike-x,0))

0    1.932523
1    1.344607
2    1.565855
3    2.479771
4    0.000000
dtype: float64

## Standard Monte Carlo Method

In [27]:
V_ave = []
V_se = []
for i in M:
    
    paths = pd.DataFrame(np.ones(i) * Price)
    for j in range(L):
         paths[j+1] = paths[j].apply(lambda x : x* np.exp((Rate - 0.5 * sigma * sigma ) * dt + sigma * np.sqrt(dt) * np.random.randn()))
    f = paths.loc[:,1:]
    final_value = f.mean(axis=1)
    V = final_value.apply(lambda x : max(Strike-x,0))  * np.exp(-Rate*Time)
    V_ave.append(V.mean())
    V_se.append(V.std() / np.sqrt(i))

V_ave = np.array(V_ave)
V_se = np.array(V_se)


st_MC = pd.DataFrame()
st_MC['Lower Bound'] =  V_ave - 1.96 * V_se
st_MC['Upper Bound'] =  V_ave + 1.96 * V_se



### Standard Monte Carlo Result

In [28]:
st_MC['length'] = st_MC['Upper Bound'] - st_MC['Lower Bound']
st_MC

,Lower Bound,Upper Bound,length
0,0.713240,0.838293,0.125052
1,0.702727,0.790720,0.087993
2,0.736175,0.799411,0.063236
3,0.745813,0.790250,0.044437
4,0.743080,0.774425,0.031346
5,0.751886,0.773914,0.022029
6,0.750038,0.765568,0.015530


## Antithetic variate method

In [31]:
V_ave = []
V_se = []
for i in M:
    
    paths = pd.DataFrame(np.ones(i) * Price)
    paths_2 = pd.DataFrame(np.ones(i) * Price)
    for j in range(L):
        z = np.random.randn(i)
        paths[j+1] = paths[j] * np.exp((Rate - 0.5 * sigma * sigma ) * dt + sigma * np.sqrt(dt) * z)
        paths_2[j+1] = paths_2[j] * np.exp((Rate - 0.5 * sigma * sigma ) * dt - sigma * np.sqrt(dt) * z)
        
    f = paths.loc[:,1:]
    final_value = f.mean(axis=1)
    V = final_value.apply(lambda x : max(Strike-x,0)) * np.exp(-Rate*Time)
    
    f = paths_2.loc[:,1:]
    final_value = f.mean(axis=1)
    V_2 = final_value.apply(lambda x : max(Strike-x,0)) * np.exp(-Rate*Time)
    
    
    V_hat = 0.5 * (V + V_2)
    V_ave.append(V_hat.mean())
    V_se.append((V_hat.std() / np.sqrt(i)))

V_ave = np.array(V_ave)
V_se = np.array(V_se)

print(V_ave)
av_method = pd.DataFrame()
av_method['Lower Bound'] =  V_ave - 1.96 * V_se
av_method['Upper Bound'] =  V_ave + 1.96 * V_se

av_method

[0.77039    0.76082471 0.75297781 0.76248361 0.75598287 0.76480291
 0.75726098]


,Lower Bound,Upper Bound
0,0.740907,0.799873
1,0.740576,0.781074
2,0.738545,0.767410
3,0.752012,0.772955
4,0.748644,0.763321
5,0.759592,0.770013
6,0.753585,0.760937


## Antithetic Variate Result

In [32]:
av_method['length'] = av_method['Upper Bound'] - av_method['Lower Bound']
av_method['Ratio_of_Width'] = st_MC['length'] / av_method['length']
av_method

,Lower Bound,Upper Bound,length,Ratio_of_Width
0,0.740907,0.799873,0.058965,2.120774
1,0.740576,0.781074,0.040498,2.172786
2,0.738545,0.767410,0.028865,2.190746
3,0.752012,0.772955,0.020944,2.121746
4,0.748644,0.763321,0.014677,2.135712
5,0.759592,0.770013,0.010421,2.113857
6,0.753585,0.760937,0.007353,2.112153


## Control variate method

### Calculating Theoritical Value Using Given Formula

In [34]:
sigma_hat = np.sqrt(sigma*sigma * (L+1)*(2*L+1) / (6*L*L))
sigma_hat

0.21466085438991292

In [35]:
mu_hat = 0.5*sigma_hat**2 + (Rate - 0.5 * sigma**2) * (L+1) / (2*L)
mu_hat

0.006112557870370371

In [36]:
d1_hat = (np.log(Price/Strike) + (mu_hat + 0.5*sigma_hat**2)*Time) / (sigma_hat * np.sqrt(Time))
d1_hat

0.13580584665484288

In [37]:
d2_hat = d1_hat - sigma_hat * np.sqrt(Time)
d2_hat

-0.07885500773507004

In [38]:
geo_asian_put = np.exp(-Rate * Time) * (-Price*np.exp(mu_hat * Time) * norm.cdf(-d1_hat) + Strike * norm.cdf(-d2_hat))
geo_asian_put

0.8025989030185081

In [40]:
V_ave = []
V_se = []
for i in M:
    
    paths = pd.DataFrame(np.ones(i) * Price)
    for j in range(L):
        z = np.random.randn(i)
        paths[j+1] = paths[j] * np.exp((Rate - 0.5 * sigma * sigma ) * dt + sigma * np.sqrt(dt) * z)
        
        
    f = paths.loc[:,1:]
    final_value = f.mean(axis=1)
    arith_value = final_value.apply(lambda x : max(Strike-x,0)) * np.exp(-Rate*Time)
    
    final_value = f.prod(axis=1) ** (1/L)
    geo_value = final_value.apply(lambda x : max(Strike-x,0)) * np.exp(-Rate*Time)
    
    V = arith_value - geo_value + geo_asian_put
    
    V_ave.append(V.mean())
    V_se.append(V.std() / np.sqrt(i))

V_ave = np.array(V_ave)
V_se = np.array(V_se)

cv_method = pd.DataFrame()
cv_method['Lower Bound'] =  V_ave - 1.96 * V_se
cv_method['Upper Bound'] =  V_ave + 1.96 * V_se

cv_method

,Lower Bound,Upper Bound
0,0.757654,0.764883
1,0.756753,0.761905
2,0.757703,0.761486
3,0.758945,0.761622
4,0.758681,0.760609
5,0.759181,0.760522
6,0.759349,0.760302


## Control Variate Method Result

In [41]:
cv_method['length'] = cv_method['Upper Bound'] - cv_method['Lower Bound']
cv_method['Ratio_of_Width'] = st_MC['length'] / cv_method['length']
cv_method

,Lower Bound,Upper Bound,length,Ratio_of_Width
0,0.757654,0.764883,0.007228,17.300241
1,0.756753,0.761905,0.005152,17.078749
2,0.757703,0.761486,0.003783,16.713719
3,0.758945,0.761622,0.002677,16.599008
4,0.758681,0.760609,0.001928,16.256454
5,0.759181,0.760522,0.001340,16.435497
6,0.759349,0.760302,0.000953,16.294183
